In [ ]:
import pandas as pd
import numpy as np
import pymongo
import warnings
import time
import random
from tqdm.notebook import tqdm
import itertools
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve, f1_score, recall_score, precision_score
from imblearn.over_sampling import SMOTE
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.regularizers import l2
import matplotlib.pyplot as plt
import seaborn as sns

warnings.filterwarnings('ignore')

# CONEXIÓN

In [ ]:
CONNECTION_STRING = "mongodb://proyectoestudiantes:suVKLoKoqjCxdeRnmZH7IICunDq54Ed33zaKcNzPBdxI2PVaohC0veT5diWHmsojaQCW6r2qohC9ACDbu5vPqQ==@proyectoestudiantes.mongo.cosmos.azure.com:10255/?ssl=true&replicaSet=globaldb&retrywrites=false&maxIdleTimeMS=120000&appName=@proyectoestudiantes@"

In [ ]:
client = pymongo.MongoClient(CONNECTION_STRING)
db = client['Estudiantes']
collection = db['Estudiantes_Materias']

datos_raw = list(collection.find({}))

registros = []
for doc in datos_raw:
    datos_personales = doc.get('datos_personales', {})
    info_academica = doc.get('academico', {})
    location = doc.get('location', {})
    info_colegio = doc.get('colegio', {})
    puntajes = doc.get('ICFES', {})
    metricas = doc.get('metricas_rendimiento', {})
    estado = doc.get('estado', {})
    periodo_info = doc.get('periodo_info', {})
    
    registro = {
        'edad': datos_personales.get('edad'),
        'genero': datos_personales.get('genero'),
        'estrato': datos_personales.get('estrato'),
        'discapacidad': datos_personales.get('discapacidad'),
        'programa': info_academica.get('programa'),
        'programa_secundario': info_academica.get('programa_secundario'),
        'tiene_programa_secundario': 1 if info_academica.get('programa_secundario') else 0,
        'semestre_actual': info_academica.get('semestre_actual'),
        'tipo_estudiante': info_academica.get('tipo_estudiante'),
        'tipo_admision': info_academica.get('tipo_admision'),
        'estado_academico': info_academica.get('estado_academico'),
        'ciudad_residencia': location.get('ciudad'),
        'depto_residencia': location.get('departamento'),
        'pais': location.get('pais'),
        'es_barranquilla': location.get('es_barranquilla'),
        'es_colombia': location.get('es_colombia'),
        'codigo_dane': location.get('codigo_dane'),
        'tipo_colegio': info_colegio.get('tipo_colegio'),
        'calendario_colegio': info_colegio.get('calendario_colegio'),
        'descripcion_bachillerato': info_colegio.get('descripcion_bachillerato'),
        'puntaje_total': puntajes.get('puntaje_total'),
        'matematicas': puntajes.get('matematicas'),
        'lectura_critica': puntajes.get('lectura_critica'),
        'sociales': puntajes.get('sociales'),
        'ciencias': puntajes.get('ciencias'),
        'ingles': puntajes.get('ingles'),
        'promedio': metricas.get('promedio_acumulado'),
        'materias_cursadas': metricas.get('materias_cursadas_total'),
        'materias_perdidas': metricas.get('materias_perdidas_total'),
        'materias_repetidas': metricas.get('materias_repetidas'),
        'perdidas_por_depto': metricas.get('materias_perdidas_por_departamento', {}),
        'beca': estado.get('becado'),
        'graduado': estado.get('graduado'),
        'desertor': estado.get('desertor'),
        'ultimo_periodo': periodo_info.get('ultimo_periodo')
    }
    registros.append(registro)

df = pd.DataFrame(registros)
df = df[df['graduado'] == False]

print(f"Datos extraidos: {df.shape[0]} estudiantes x {df.shape[1]} variables")
print(f"Desertores: {df['desertor'].sum()} ({df['desertor'].sum()/len(df)*100:.2f}%)")

client.close()

# Preprocesamiento

In [ ]:
perdidas_df = pd.json_normalize(df['perdidas_por_depto'])
perdidas_df = perdidas_df.add_prefix('perdidas_')
df = pd.concat([df.drop('perdidas_por_depto', axis=1), perdidas_df], axis=1)

cols_perdidas_depto = [col for col in df.columns if col.startswith('perdidas_')]

df_desertores = df[df['desertor'] == 1].copy()
df_no_desertores = df[df['desertor'] == 0].copy()

numericas = df.select_dtypes(include=[np.number]).columns.tolist()
numericas = [col for col in numericas if col not in ['desertor', 'graduado']]

for col in numericas:
    if df[col].isna().sum() > 0:
        if col in cols_perdidas_depto or col == 'semestre_actual':
            # Llenar con 0: perdidas por departamento y semestre actual vacio
            df[col] = df[col].fillna(0)
        else:
            mediana_desertores = df_desertores[col].median()
            mediana_no_desertores = df_no_desertores[col].median()
            df.loc[(df['desertor'] == 1) & (df[col].isna()), col] = mediana_desertores
            df.loc[(df['desertor'] == 0) & (df[col].isna()), col] = mediana_no_desertores

categoricas = ['genero', 'discapacidad', 'programa', 'programa_secundario', 
               'tipo_estudiante', 'tipo_admision', 'estado_academico',
               'ciudad_residencia', 'depto_residencia', 'pais', 
               'tipo_colegio', 'calendario_colegio', 'descripcion_bachillerato',
               'ultimo_periodo']

for col in categoricas:
    if col in df.columns and df[col].isna().sum() > 0:
        moda_desertores = df[df['desertor'] == 1][col].mode()
        moda_no_desertores = df[df['desertor'] == 0][col].mode()
        valor_desertores = moda_desertores[0] if len(moda_desertores) > 0 else 'Desconocido'
        valor_no_desertores = moda_no_desertores[0] if len(moda_no_desertores) > 0 else 'Desconocido'
        df.loc[(df['desertor'] == 1) & (df[col].isna()), col] = valor_desertores
        df.loc[(df['desertor'] == 0) & (df[col].isna()), col] = valor_no_desertores

encoders = {}
for col in categoricas:
    if col in df.columns:
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col].astype(str))
        encoders[col] = le

cols_object = df.select_dtypes(include=['object']).columns.tolist()
cols_object = [col for col in cols_object if col not in ['desertor', 'graduado']]

if cols_object:
    for col in cols_object:
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col].astype(str))
        encoders[col] = le

print(f"Datos procesados: {df.shape[0]} filas x {df.shape[1]} columnas")

Cargando datos...
Total filas: 10226
Total columnas: 30

Distribución de desertor:
desertor
0    9749
1     477
Name: count, dtype: int64
Tasa deserción: 4.66%

Procesando perdidas por departamento...
Categorías de materias encontradas: {'Dpto. Derecho', 'Dpto. Lenguas Extranjeras', 'Dpto. Español', 'Dpto. de Economía', 'Dpto. Humanidades y Filosofía', 'Dpto. Psicología', 'Dpto. Química y Biología', 'Dpto. Salud Pública', 'Dpto. Ing. Civil y Ambiental', 'Dpto. Odontología', 'Dpto. Cs Politica y Rel Intern', 'Dpto. Historia y Cs. Sociales', 'Dpto. Ingeniería de Sistemas', 'Dpto. Ingeniería Mecánica', 'Dpto. Finanzas y Contaduría', 'Dpto. Diseño', 'Dpto. Educación', 'Dpto. Arquitectura y Urbanismo', 'Dpto. Ingeniería Industrial', 'Dpto. Matematicas y estadístic', 'Dpto. Emprendim y Management', 'Dpto. Comunicación Social', 'Dpto. Música', 'Dpto. Física', 'Dpto.Ing Eléctrica-Electrónica', 'Dpto. Enfermería', 'Dpto. Mercadeo y Neg. Internac', 'Dpto. Medicina'}

Rellenando valores nulos...


C:\Users\claud\AppData\Local\Temp\ipykernel_20476\1458097616.py:51: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].median(), inplace=True)
C:\Users\claud\AppData\Local\Temp\ipykernel_20476\1458097616.py:60: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For exa

# partición

In [ ]:
df = df.dropna(subset=['desertor'])

X = df.drop(['desertor', 'graduado'], axis=1)
y = df['desertor'].astype(int)

# Primera división: 70% entrenamiento, 30% temporal
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

# Segunda división: dividir el 30% temporal en 15% validación y 15% prueba
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp
)

# Escalar datos
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

print(f"Train: {X_train_scaled.shape[0]} ({X_train_scaled.shape[0]/len(df)*100:.1f}%)")
print(f"Validacion: {X_val_scaled.shape[0]} ({X_val_scaled.shape[0]/len(df)*100:.1f}%)")
print(f"Test: {X_test_scaled.shape[0]} ({X_test_scaled.shape[0]/len(df)*100:.1f}%)")
print(f"Features: {X_train_scaled.shape[1]}")
print(f"\nDistribucion desertores:")
print(f"Train: {y_train.sum()} ({y_train.sum()/len(y_train)*100:.1f}%)")
print(f"Validacion: {y_val.sum()} ({y_val.sum()/len(y_val)*100:.1f}%)")
print(f"Test: {y_test.sum()} ({y_test.sum()/len(y_test)*100:.1f}%)")

# Configuración grid search 

In [ ]:
# Diccionarios de configuracion para grid search
tecnicas_muestreo = {
    'original': None,
    'smote_30': SMOTE(sampling_strategy=0.30, random_state=42),
    'smote_40': SMOTE(sampling_strategy=0.40, random_state=42),
    'smote_50': SMOTE(sampling_strategy=0.50, random_state=42),
}

class_weights_base = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),
    y=y_train
)
cw_base_dict = {0: class_weights_base[0], 1: class_weights_base[1]}

configuraciones_pesos = {
    'balanced': cw_base_dict,
    'high_recall_2x': {0: cw_base_dict[0], 1: cw_base_dict[1] * 2},
    'high_recall_3x': {0: cw_base_dict[0], 1: cw_base_dict[1] * 3},
    'moderate': {0: 1.0, 1: 10.0},
    'none': None
}

arquitecturas = {
    'medium_1': [256, 128, 64],
    'medium_2': [256, 128, 64, 32],
    'medium_3': [512, 256, 128],
}

hiperparametros = {
    'dropout_rates': [0.2, 0.3, 0.4],
    'learning_rates': [0.001, 0.0005],
    'batch_sizes': [32, 64],
    'optimizers': ['adam', 'rmsprop'],
}

regularizaciones = {
    'none': None,
    'l2_light': l2(0.001),
    'l2_medium': l2(0.01),
}

thresholds = [0.3, 0.35, 0.4, 0.45, 0.5]



In [ ]:
# Generar todas las combinaciones usando los diccionarios
random.seed(42)
np.random.seed(42)

configuraciones_entrenamiento = []

# Recorrer cada diccionario para generar combinaciones
for muestreo_nombre, muestreo_obj in tecnicas_muestreo.items():
    for peso_nombre, peso_obj in configuraciones_pesos.items():
        for arq_nombre, capas in arquitecturas.items():
            for dropout in hiperparametros['dropout_rates']:
                for lr in hiperparametros['learning_rates']:
                    for bs in hiperparametros['batch_sizes']:
                        for opt in hiperparametros['optimizers']:
                            for reg_nombre, reg_obj in regularizaciones.items():
                                config = {
                                    'muestreo_nombre': muestreo_nombre,
                                    'muestreo': muestreo_obj,
                                    'peso_nombre': peso_nombre,
                                    'pesos': peso_obj,
                                    'arquitectura_nombre': arq_nombre,
                                    'capas': capas,
                                    'dropout': dropout,
                                    'learning_rate': lr,
                                    'batch_size': bs,
                                    'optimizer': opt,
                                    'regularizacion_nombre': reg_nombre,
                                    'regularizacion': reg_obj,
                                }
                                configuraciones_entrenamiento.append(config)


# Limitar a 500 configuraciones aleatorias si hay muchas
if len(configuraciones_entrenamiento) > 500:
    random.shuffle(configuraciones_entrenamiento)
    configuraciones_entrenamiento = configuraciones_entrenamiento[:500]

print(f"Configuraciones generadas: {len(configuraciones_entrenamiento)}")


In [ ]:
# Entrenar todos los modelos


resultados_busqueda = []
modelos_entrenados = {}

early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, verbose=0)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.00001, verbose=0)

print(f"Entrenando {len(configuraciones_entrenamiento)} modelos...")
tiempo_inicio_global = time.time()

# Barra de progreso
pbar = tqdm(enumerate(configuraciones_entrenamiento, 1), total=len(configuraciones_entrenamiento), 
            desc="Entrenando modelos", unit="modelo")

for idx, config in pbar:
    try:
        # Aplicar tecnica de muestreo
        if config['muestreo'] is not None:
            X_train_prep, y_train_prep = config['muestreo'].fit_resample(X_train_scaled, y_train)
        else:
            X_train_prep = X_train_scaled
            y_train_prep = y_train
        
        # Construir modelo
        modelo = Sequential()
        modelo.add(Dense(config['capas'][0], activation='relu', input_dim=X_train_prep.shape[1], 
                        kernel_regularizer=config['regularizacion']))
        modelo.add(Dropout(config['dropout']))
        
        for units in config['capas'][1:]:
            modelo.add(Dense(units, activation='relu', kernel_regularizer=config['regularizacion']))
            modelo.add(Dropout(config['dropout']))
        
        modelo.add(Dense(1, activation='sigmoid'))
        
        # Configurar optimizador
        if config['optimizer'] == 'adam':
            opt = Adam(learning_rate=config['learning_rate'])
        else:
            opt = RMSprop(learning_rate=config['learning_rate'])
        
        modelo.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
        
        # Entrenar
        inicio = time.time()
        history = modelo.fit(
            X_train_prep, y_train_prep, epochs=100, batch_size=config['batch_size'],
            validation_data=(X_val_scaled, y_val), class_weight=config['pesos'], 
            callbacks=[early_stop, reduce_lr], verbose=0
        )
        tiempo_entrenamiento = time.time() - inicio
        
        # Predecir en validación
        y_pred_proba = modelo.predict(X_val_scaled, verbose=0).flatten()
        
        # Guardar modelo
        modelo_id = f"modelo_{idx}"
        modelos_entrenados[modelo_id] = {
            'modelo': modelo,
            'config': config,
            'y_pred_proba': y_pred_proba,
            'history': history
        }
        
        # Evaluar con diferentes thresholds
        for thresh in thresholds:
            y_pred = (y_pred_proba >= thresh).astype(int)
            
            recall = recall_score(y_val, y_pred)
            precision = precision_score(y_val, y_pred, zero_division=0)
            f1 = f1_score(y_val, y_pred)
            auc = roc_auc_score(y_val, y_pred_proba)
            
            cm = confusion_matrix(y_val, y_pred)
            tn, fp, fn, tp = cm.ravel()
            
            score_custom = 0.6 * recall + 0.3 * auc + 0.1 * precision
            
            resultado = {
                'modelo_id': modelo_id, 'config_idx': idx,
                'muestreo': config['muestreo_nombre'], 'pesos': config['peso_nombre'],
                'arquitectura': config['arquitectura_nombre'], 'capas': str(config['capas']),
                'dropout': config['dropout'], 'learning_rate': config['learning_rate'],
                'batch_size': config['batch_size'], 'optimizer': config['optimizer'],
                'regularizacion': config['regularizacion_nombre'], 'threshold': thresh,
                'recall': recall, 'precision': precision, 'f1': f1, 'auc': auc,
                'score_custom': score_custom, 'tp': tp, 'fp': fp, 'tn': tn, 'fn': fn,
                'epochs': len(history.history['loss']), 'tiempo': tiempo_entrenamiento
            }
            resultados_busqueda.append(resultado)
        
        # Actualizar barra de progreso
        tiempo_transcurrido = time.time() - tiempo_inicio_global
        tiempo_restante = (tiempo_transcurrido / idx) * (len(configuraciones_entrenamiento) - idx)
        pbar.set_postfix({
            'Tiempo': f'{tiempo_transcurrido/60:.1f}min',
            'Restante': f'~{tiempo_restante/60:.1f}min',
            'Modelos': len(modelos_entrenados)
        })
        
    except Exception as e:
        pbar.write(f"Error en modelo {idx}: {str(e)}")
        continue

pbar.close()

tiempo_total = time.time() - tiempo_inicio_global
print(f"\nEntrenamiento completado en {tiempo_total/60:.2f} minutos")
print(f"Modelos: {len(modelos_entrenados)} | Evaluaciones: {len(resultados_busqueda)}")

In [ ]:
# Analizar resultados - Solo modelos con recall > 75%
df_resultados = pd.DataFrame(resultados_busqueda)

# Filtrar solo modelos con recall mayor a 75%
df_high_recall = df_resultados[df_resultados['recall'] > 0.75].copy()

print("ANALISIS DE RESULTADOS - MODELOS CON RECALL > 75%")
print(f"\nTotal de evaluaciones: {len(df_resultados)}")
print(f"Evaluaciones con recall > 75%: {len(df_high_recall)} ({len(df_high_recall)/len(df_resultados)*100:.1f}%)")

if len(df_high_recall) > 0:
    print(f"\nEstadisticas de modelos con recall > 75%:")
    print(f"Recall - Media: {df_high_recall['recall'].mean():.3f} | Max: {df_high_recall['recall'].max():.3f}")
    print(f"Precision - Media: {df_high_recall['precision'].mean():.3f} | Max: {df_high_recall['precision'].max():.3f}")
    print(f"F1 - Media: {df_high_recall['f1'].mean():.3f} | Max: {df_high_recall['f1'].max():.3f}")
    print(f"AUC - Media: {df_high_recall['auc'].mean():.3f} | Max: {df_high_recall['auc'].max():.3f}")
    
    print(f"\n{'='*120}")
    print(f"TOP 20 MODELOS CON RECALL > 75% (ordenados por score personalizado)")
    print(f"{'='*120}")
    top_high_recall = df_high_recall.nlargest(10, 'score_custom')
    print(top_high_recall[['modelo_id', 'muestreo', 'pesos', 'arquitectura', 'dropout', 'learning_rate', 
                            'batch_size', 'optimizer', 'threshold', 'recall', 'precision', 'f1', 'auc', 
                            'score_custom', 'tp', 'fp', 'fn']].to_string(index=False))
    
    print(f"\n{'='*120}")
    print(f"MEJOR BALANCE (TOP 10 por Precision entre modelos con Recall > 75%)")
    print(f"{'='*120}")
    top_precision = df_high_recall.nlargest(10, 'precision')
    print(top_precision[['modelo_id', 'muestreo', 'pesos', 'arquitectura', 'threshold', 
                         'recall', 'precision', 'f1', 'auc', 'tp', 'fp', 'fn']].to_string(index=False))
    
    print(f"\n{'='*120}")
    print(f"MAXIMA DETECCION (TOP 10 por Recall)")
    print(f"{'='*120}")
    top_recall = df_high_recall.nlargest(10, 'recall')
    print(top_recall[['modelo_id', 'muestreo', 'pesos', 'arquitectura', 'threshold', 
                      'recall', 'precision', 'f1', 'auc', 'tp', 'fp', 'fn']].to_string(index=False))
    
    print(f"\n{'='*120}")
    print(f"MEJOR AUC (TOP 10)")
    print(f"{'='*120}")
    top_auc = df_high_recall.nlargest(10, 'auc')
    print(top_auc[['modelo_id', 'muestreo', 'pesos', 'arquitectura', 'threshold', 
                   'recall', 'precision', 'f1', 'auc', 'tp', 'fp', 'fn']].to_string(index=False))
else:
    print("\nNo se encontraron modelos con recall mayor a 75%")
    print("\nMejores modelos disponibles:")
    print(df_resultados.nlargest(10, 'recall')[['modelo_id', 'recall', 'precision', 'f1', 'auc']].to_string(index=False))

In [ ]:
# Visualizar mejor modelo
mejor_config = df_resultados.nlargest(1, 'score_custom').iloc[0]
mejor_modelo_id = mejor_config['modelo_id']
mejor_modelo_info = modelos_entrenados[mejor_modelo_id]

print("MEJOR MODELO")
print(f"\nModelo: {mejor_modelo_id}")
print(f"Muestreo: {mejor_config['muestreo']} | Pesos: {mejor_config['pesos']}")
print(f"Arquitectura: {mejor_config['arquitectura']} {mejor_config['capas']}")
print(f"Threshold: {mejor_config['threshold']} | Dropout: {mejor_config['dropout']} | LR: {mejor_config['learning_rate']}")
print(f"\nMetricas:")
print(f"Recall: {mejor_config['recall']:.3f} | Precision: {mejor_config['precision']:.3f}")
print(f"F1-Score: {mejor_config['f1']:.3f} | AUC: {mejor_config['auc']:.3f}")
print(f"\nMatriz de confusion:")
print(f"TP: {mejor_config['tp']} | FP: {mejor_config['fp']} | TN: {mejor_config['tn']} | FN: {mejor_config['fn']}")

fig, axes = plt.subplots(2, 2, figsize=(15, 12))

y_pred_mejor = (mejor_modelo_info['y_pred_proba'] >= mejor_config['threshold']).astype(int)
cm = confusion_matrix(y_val, y_pred_mejor)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[0, 0])
axes[0, 0].set_title(f'Matriz de Confusion\nRecall: {mejor_config["recall"]:.3f} | Precision: {mejor_config["precision"]:.3f}')
axes[0, 0].set_ylabel('Verdadero')
axes[0, 0].set_xlabel('Predicho')
axes[0, 0].set_xticklabels(['No Desertor', 'Desertor'])
axes[0, 0].set_yticklabels(['No Desertor', 'Desertor'])

fpr, tpr, thresholds_roc = roc_curve(y_val, mejor_modelo_info['y_pred_proba'])
axes[0, 1].plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC (AUC = {mejor_config["auc"]:.3f})')
axes[0, 1].plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random')
axes[0, 1].set_xlim([0.0, 1.0])
axes[0, 1].set_ylim([0.0, 1.05])
axes[0, 1].set_xlabel('Tasa de Falsos Positivos')
axes[0, 1].set_ylabel('Tasa de Verdaderos Positivos (Recall)')
axes[0, 1].set_title('Curva ROC')
axes[0, 1].legend(loc="lower right")
axes[0, 1].grid(True, alpha=0.3)

axes[1, 0].hist(mejor_modelo_info['y_pred_proba'][y_val == 0], bins=50, alpha=0.5, label='No Desertor', color='blue')
axes[1, 0].hist(mejor_modelo_info['y_pred_proba'][y_val == 1], bins=50, alpha=0.5, label='Desertor', color='red')
axes[1, 0].axvline(mejor_config['threshold'], color='green', linestyle='--', linewidth=2, label=f'Threshold={mejor_config["threshold"]:.2f}')
axes[1, 0].set_xlabel('Probabilidad predicha')
axes[1, 0].set_ylabel('Frecuencia')
axes[1, 0].set_title('Distribucion de Probabilidades Predichas')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

thresholds_test = [0.3, 0.35, 0.4, 0.45, 0.5]
recalls = []
precisions = []
f1s = []

for thresh in thresholds_test:
    y_pred_temp = (mejor_modelo_info['y_pred_proba'] >= thresh).astype(int)
    recalls.append(recall_score(y_val, y_pred_temp))
    precisions.append(precision_score(y_val, y_pred_temp, zero_division=0))
    f1s.append(f1_score(y_val, y_pred_temp))

axes[1, 1].plot(thresholds_test, recalls, 'o-', label='Recall', linewidth=2, markersize=8)
axes[1, 1].plot(thresholds_test, precisions, 's-', label='Precision', linewidth=2, markersize=8)
axes[1, 1].plot(thresholds_test, f1s, '^-', label='F1-Score', linewidth=2, markersize=8)
axes[1, 1].axvline(mejor_config['threshold'], color='red', linestyle='--', alpha=0.5, label=f'Seleccionado={mejor_config["threshold"]:.2f}')
axes[1, 1].set_xlabel('Threshold')
axes[1, 1].set_ylabel('Metrica')
axes[1, 1].set_title('Metricas vs Threshold')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)
axes[1, 1].set_ylim([0, 1])

plt.tight_layout()
plt.show()

In [ ]:
# Generar predicciones
mejores_probabilidades = mejor_modelo_info['y_pred_proba']

df_predicciones = pd.DataFrame({
    'indice_original': X_val.index,
    'probabilidad_desercion': mejores_probabilidades,
    'desertor_real': y_val.values,
    'prediccion': y_pred_mejor
})

df_predicciones = df_predicciones.sort_values('probabilidad_desercion', ascending=False)

print("TOP 20 ESTUDIANTES CON MAYOR RIESGO DE DESERCION")
print(df_predicciones.head(20).to_string(index=False))

estudiantes_en_riesgo = df_predicciones[df_predicciones['probabilidad_desercion'] >= mejor_config['threshold']]
print(f"\nRESUMEN")
print(f"Estudiantes evaluados: {len(df_predicciones)}")
print(f"Identificados en riesgo: {len(estudiantes_en_riesgo)} ({len(estudiantes_en_riesgo)/len(df_predicciones)*100:.2f}%)")
print(f"Verdaderos positivos: {estudiantes_en_riesgo['desertor_real'].sum()}")
print(f"Falsos positivos: {(estudiantes_en_riesgo['desertor_real'] == 0).sum()}")

print(f"\nDISTRIBUCION DE PROBABILIDADES")
rangos = [
    (0.0, 0.2, 'Riesgo muy bajo'),
    (0.2, 0.4, 'Riesgo bajo'),
    (0.4, 0.6, 'Riesgo moderado'),
    (0.6, 0.8, 'Riesgo alto'),
    (0.8, 1.0, 'Riesgo muy alto')
]

for inicio, fin, etiqueta in rangos:
    count = ((df_predicciones['probabilidad_desercion'] >= inicio) & 
             (df_predicciones['probabilidad_desercion'] < fin)).sum()
    pct = count / len(df_predicciones) * 100
    print(f"{etiqueta} [{inicio:.1f}-{fin:.1f}): {count} estudiantes ({pct:.2f}%)")

print("=" * 70)
print("EVALUACIÓN DEL MODELO CON MEJORAS")
print("=" * 70)

# PASO 1: Cargar modelo y datos de test
print("\n1. Cargando modelo y datos...")
model = tf.keras.models.load_model('modelo_desercion.h5', compile=False)

X_test = np.load('X_test.npy')
y_test = np.load('y_test.npy')

print(f"Modelo cargado")
print(f"X_test shape: {X_test.shape}")
print(f"y_test shape: {y_test.shape}")

# PASO 2: Hacer predicciones con THRESHOLD AJUSTADO
print("\n2. Haciendo predicciones...")
y_pred_proba = model.predict(X_test)

# MEJORA 1: Threshold más bajo para detectar más desertores
THRESHOLD = 0.3  # Bajado de 0.5 a 0.3
y_pred = (y_pred_proba > THRESHOLD).astype(int).flatten()

print(f"Predicciones completadas con threshold = {THRESHOLD}")

# PASO 3: Métricas generales
print("\n3. MÉTRICAS DE EVALUACIÓN:")
print("\nReporte de clasificación:")
print(classification_report(y_test, y_pred, target_names=['No Desertor', 'Desertor']))

# ROC-AUC
auc_score = roc_auc_score(y_test, y_pred_proba)
print(f"\nROC-AUC Score: {auc_score:.4f}")

# PASO 4: Matriz de confusión
print("\n4. Matriz de Confusión:")
cm = confusion_matrix(y_test, y_pred)
print(cm)

# Calcular métricas específicas
tn, fp, fn, tp = cm.ravel()
print(f"\nDesertores detectados: {tp} de {tp+fn} ({tp/(tp+fn)*100:.2f}%)")
print(f"Falsos positivos: {fp}")
print(f"Verdaderos negativos: {tn}")

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['No Desertor', 'Desertor'],
            yticklabels=['No Desertor', 'Desertor'])
plt.title(f'Matriz de Confusión (threshold={THRESHOLD})')
plt.ylabel('Real')
plt.xlabel('Predicción')
plt.tight_layout()
plt.savefig('matriz_confusion_mejorada.png', dpi=300)
print("\nMatriz guardada: matriz_confusion_mejorada.png")

# PASO 5: Curva ROC
print("\n5. Curva ROC:")
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, linewidth=2, label=f'ROC (AUC = {auc_score:.4f})')
plt.plot([0, 1], [0, 1], 'k--', linewidth=1, label='Random')
plt.scatter([fp/(fp+tn)], [tp/(tp+fn)], color='red', s=100, zorder=5, 
            label=f'Threshold={THRESHOLD}')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate (Recall)')
plt.title('Curva ROC - Modelo Mejorado')
plt.legend(loc="lower right")
plt.grid(alpha=0.3)
plt.tight_layout()
plt.savefig('curva_roc_mejorada.png', dpi=300)
print("Curva ROC guardada: curva_roc_mejorada.png")

# PASO 6: Análisis de threshold óptimo
print("\n6. ANÁLISIS DE DIFERENTES THRESHOLDS:")
print("-" * 70)
for thresh in [0.2, 0.3, 0.4, 0.5]:
    y_pred_temp = (y_pred_proba > thresh).astype(int).flatten()
    cm_temp = confusion_matrix(y_test, y_pred_temp)
    tn_t, fp_t, fn_t, tp_t = cm_temp.ravel()
    
    recall = tp_t / (tp_t + fn_t)
    precision = tp_t / (tp_t + fp_t) if (tp_t + fp_t) > 0 else 0
    
    print(f"Threshold {thresh}: Recall={recall:.3f} | Precision={precision:.3f} | Detectados={tp_t}/{tp_t+fn_t}")

print("\n" + "=" * 70)
print("¡EVALUACIÓN COMPLETADA!")
print("=" * 70)
